In [ ]:
# 第五章作业，这里直接模拟了调取网络接口查取数据，以及使用向量数据库的方式

In [ ]:
# 首先定义三个工具：1.网络接口，2.向量数据库，3.表情包生成
# 模拟三个主题文案：1.海参文案，2.美妆文案，3.创业文案

In [1]:
#工具一：这里使用了天聚数行的文案生成接口，但是实际效果其实不如直接使用deepseek自带的的网络爬虫+LLM，这里只是为了模拟它当成一个查询真实数据的接口,充当deepseek的网络爬虫阶段。
import requests
import os
def query_web(query: str) -> str:
    """
    调用天行数据文案生成API，根据关键词/主题生成短文案

    :param api_key: 你的天行API密钥
    :param text: 文案主题、关键词、重点（不超过60字）
    :return: 生成的短文案内容，失败时返回错误信息
    """
    url = "https://apis.tianapi.com/aitextcall/index"
    payload = {
        "key": os.getenv("TIANXING_API"),
        "text": query
    }
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    try:
        resp = requests.post(url, data=payload, headers=headers, timeout=10)
        data = resp.json()
        if data["code"] == 200:
            # 文案内容在 result['content']
            return data["result"]["content"]
        else:
            return f"接口调用失败，错误信息：{data.get('msg', '未知错误')}"
    except Exception as e:
        return f"请求异常：{e}"



In [2]:
# 工具二 模拟向量数据库

from pymilvus import model as milvus_model
from pymilvus import MilvusClient
from tqdm import tqdm
#创建相似度函数
embedding_model = milvus_model.DefaultEmbeddingFunction()

# 创建数据库客户端
milvus_client = MilvusClient(uri="./milvus_demo.db")


collection_name = "rednote"
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

#创建向量数据库集合rednote
milvus_client.create_collection(
collection_name = collection_name,
dimension = 768,
metric_type = "IP",  # 内积距离
consistency_level = "Strong")

# 模拟插入数据，对海参、美妆、创业每个主题进行了10条数据的插入.为了模拟方便, 分别调用工具一的接口函数来生成。
topics = ["海参","美妆","创业"]
text_lines = [query_web(i) for _ in range(10) for i in topics]
# print(text_lines)

data = []
doc_embeddings = embedding_model.encode_documents(text_lines)
for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})
milvus_client.insert(collection_name=collection_name, data=data)

def query_database(question):
    search_res = milvus_client.search(
    collection_name = collection_name,
    data = embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit = 3,  # 返回前3个结果
    search_params = {"metric_type": "IP", "params": {}},  # 内积距离
    output_fields = ["text"],  # 返回 text 字段
    )
    retrieved_lines= [
    res["entity"]["text"]
    for res in search_res[0]
    ]
    context = "\n".join(retrieved_lines)

    return context

/home/ubuntu/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 228365.01it/s]


In [3]:
# 工具三, 生成表情包，由于表情包有限，可以直接写死在代码中
import random
import time

def generate_emoji(context: str) -> list:
    """根据主题内容生成丰富/有趣的表情包"""
    # print(f"[Tool Call] 模拟生成表情包，上下文：{context}")
    # time.sleep(0.1) # 模拟延迟

    # 针对不同主题设定不同的表情库
    topic_emoji_map = {
        "海参": [
            "🌊", "🐟", "🦑", "🦞", "🌱", "💪", "🧂", "🍲", "🥢", "😋", "👏", "✨", "🧜‍♂️"
        ],
        "美妆": [
            "💄", "💅", "👑", "✨", "💖", "👗", "🌸", "😍", "🪞", "🧴", "🛍️", "🧚‍♀️", "🌈"
        ],
        "创业": [
            "🚀", "💡", "📈", "🤝", "🏆", "📊", "💸", "🏢", "📝", "👏", "🧑‍💻", "🔝", "🔥"
        ]
    }

    # 默认表情备选
    default_emoji = ["✨", "👍", "🎉", "💯", "😎", "🥰", "🙌", "🌟"]

    # 简单规则：根据 context 里的主题词分配表情
    for topic, emoji_list in topic_emoji_map.items():
        if topic in context:
            return random.sample(emoji_list, 5)
    # 没命中时，返回默认表情
    return random.sample(default_emoji, 5)

# ====== 示例调用 ======
# docs = [
#     "海参富含蛋白质，是健康养生的好选择",
#     "美妆新趋势，打造精致妆容",
#     "创业路上，坚持和创新同样重要"
# ]
#
# for doc in docs:
#     emoji_pack = mock_generate_emoji(doc)
#     print(f"文案：{doc}\n表情包：{emoji_pack}\n")

In [4]:
available_tools = {
    "query_web": query_web,
    "query_database": query_database,
    "generate_emoji": generate_emoji,
}

In [5]:
# 定义工具说明书
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "query_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要查询的关键词，如海参、美妆、创业"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "要查询的关键词，比如：海参、美妆、创业"
                    }
                },
                "required": ["question"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

In [10]:
# 开始deepseek调用，后续和课上示例代码一致了
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

# 初始化 DeepSeek 客户端

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [11]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。尽可能多的调用工具，把每一个可用的工具都调用一次。
"""

In [12]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。

    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。

    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """

    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")

    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]

    iteration_count = 0
    final_response = None

    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")

        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION, # 告知模型可用的工具
                tool_choice="auto" # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message

            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史

                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")

                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史

            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")

                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)

                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break

        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break

    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

In [13]:
# 实际输出
topic1 = "海参"
tone_style_1 = "高端大气上档次"
result_1 = generate_rednote(topic1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：海参，风格：高端大气上档次

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_database'，参数：{'question': '海参'}
Observation: 工具返回结果："拥抱大海，享受大自然的馈赠。"
"一叶扁舟载我行，畅游在生命的海洋中。"
"海参生长在湿润的地方，它坚韧不拔，生命力无比顽强。"
"海参不仅是食物，更是生活的必需品。"
"每个人都有自己的小世界，选择自己热爱的食物，才是生活中的正确态度。"
"海参虽小，却富含丰富的营养和价值，值得我们去珍视和品尝。"
"在夜晚的星空下，你的美丽如同繁星点点，闪烁着独特的魅力。让我们一起用化妆品为你的脸庞添加亮色和光彩，让每一刻都成为你独一无二的故事。在每个人的化妆故事中，每一天都是一个值得纪念的日子。今天，就让我来分享一款专为你打造的美容产品——DGH3e，让你拥有光彩照人的美。无论你是正式场合还是休闲约会，它都能帮助你在人群中脱颖而出。立即购买，开始你的美丽旅程吧！"
"创业，是一场持续探索未知、突破自我、追求梦想的旅程。在这个时代，有无数的可能性等待着我们去挖掘和发现。让我们一起，在创业的路上，勇敢前行，披荆斩棘，挑战自我，实现我们的价值。因为，只有创业，我们才能真正找到属于自己的舞台，实现自我价值的最大化。"
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '高端海参，营养丰富，滋补养生'}
Observation: 工具返回结果：['🌊', '🦞', '👏', '😋', '🦑']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "🌊深海珍品｜海参的滋补秘密，让你吃出健康与高贵！",
  "body": "谁说滋补一定要苦口？这款深海珍品——海参，不仅口感鲜美，更是营养界的王者！\n\n每一口都是大自然的馈赠，富含胶原蛋白、氨基酸和多种微量元素，让你从内到外焕发光彩✨。无论是煲汤、炖煮还是凉拌，都能轻松驾驭，满足你对高端食材的所有幻想！\n\n🦞 为什么选择海参？\n- 高蛋白低脂肪，健康滋补两不误\n- 提升免疫力，

In [14]:
topic2 = "美妆"
tone_style_2 = "活泼甜美"
result_2 = generate_rednote(topic2, tone_style_2)
print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美妆，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_database'，参数：{'question': '美妆'}
Observation: 工具返回结果："拥抱大海，享受大自然的馈赠。"
"一叶扁舟载我行，畅游在生命的海洋中。"
"海参生长在湿润的地方，它坚韧不拔，生命力无比顽强。"
"海参不仅是食物，更是生活的必需品。"
"每个人都有自己的小世界，选择自己热爱的食物，才是生活中的正确态度。"
"海参虽小，却富含丰富的营养和价值，值得我们去珍视和品尝。"
"在夜晚的星空下，你的美丽如同繁星点点，闪烁着独特的魅力。让我们一起用化妆品为你的脸庞添加亮色和光彩，让每一刻都成为你独一无二的故事。在每个人的化妆故事中，每一天都是一个值得纪念的日子。今天，就让我来分享一款专为你打造的美容产品——DGH3e，让你拥有光彩照人的美。无论你是正式场合还是休闲约会，它都能帮助你在人群中脱颖而出。立即购买，开始你的美丽旅程吧！"
"创业，是一场持续探索未知、突破自我、追求梦想的旅程。在这个时代，有无数的可能性等待着我们去挖掘和发现。让我们一起，在创业的路上，勇敢前行，披荆斩棘，挑战自我，实现我们的价值。因为，只有创业，我们才能真正找到属于自己的舞台，实现自我价值的最大化。"
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '美妆产品推荐'}
Observation: 工具返回结果：['🪞', '🧚\u200d♀️', '🌸', '✨', '🌈']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "✨【美妆种草】这款神仙产品让我秒变仙女！🧚‍♀️",
  "body": "姐妹们！今天我要分享一款让我惊艳到尖叫的美妆神器！🌸\n\n最近入手了这款DGH3e，简直是化妆包里的宝藏！无论是日常通勤还是约会派对，它都能让我瞬间拥有水润透亮的肌肤，妆容服帖到不行！🌈\n\n🌟 亮点：\n- 超强持妆力，一整天不脱妆！\n- 轻薄质地，完全不会闷痘！\n- 提亮肤色，自带柔焦效果！\n\n用了它之后，

In [16]:
topic3 = "创业"
tone_style_3 = "热情励志"
result_3 = generate_rednote(topic3, tone_style_3)
print("\n--- 生成的文案 3 ---")
print(result_3)




🚀 启动小红书文案生成助手，产品：创业，风格：热情励志

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_database'，参数：{'question': '创业'}
Observation: 工具返回结果："拥抱大海，享受大自然的馈赠。"
"一叶扁舟载我行，畅游在生命的海洋中。"
"海参生长在湿润的地方，它坚韧不拔，生命力无比顽强。"
"海参不仅是食物，更是生活的必需品。"
"每个人都有自己的小世界，选择自己热爱的食物，才是生活中的正确态度。"
"海参虽小，却富含丰富的营养和价值，值得我们去珍视和品尝。"
"在夜晚的星空下，你的美丽如同繁星点点，闪烁着独特的魅力。让我们一起用化妆品为你的脸庞添加亮色和光彩，让每一刻都成为你独一无二的故事。在每个人的化妆故事中，每一天都是一个值得纪念的日子。今天，就让我来分享一款专为你打造的美容产品——DGH3e，让你拥有光彩照人的美。无论你是正式场合还是休闲约会，它都能帮助你在人群中脱颖而出。立即购买，开始你的美丽旅程吧！"
"创业，是一场持续探索未知、突破自我、追求梦想的旅程。在这个时代，有无数的可能性等待着我们去挖掘和发现。让我们一起，在创业的路上，勇敢前行，披荆斩棘，挑战自我，实现我们的价值。因为，只有创业，我们才能真正找到属于自己的舞台，实现自我价值的最大化。"
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '创业励志'}
Observation: 工具返回结果：['📊', '🚀', '🧑\u200d💻', '💸', '🤝']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "创业路上，勇敢追梦！🚀 从0到1的蜕变之旅",
  "body": "创业，是一场持续探索未知、突破自我、追求梦想的旅程！📊\n\n在这个充满可能性的时代，我们每个人都可以成为自己命运的主宰者。无论是从零开始的小白，还是已经踏上征途的创业者，每一步都值得被记录和分享！🧑‍💻\n\n💡 我的创业心得：\n1. **勇敢尝试**：不要害怕失败，每一次跌倒都是成长的垫脚石。\n2. **持续学习**：市场瞬息

In [17]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [18]:
# 格式化
print(format_rednote_for_markdown(result_1))
print("####################################")
print(format_rednote_for_markdown(result_2))
print("####################################")
print(format_rednote_for_markdown(result_3))

## 🌊深海珍品｜海参的滋补秘密，让你吃出健康与高贵！

谁说滋补一定要苦口？这款深海珍品——海参，不仅口感鲜美，更是营养界的王者！

每一口都是大自然的馈赠，富含胶原蛋白、氨基酸和多种微量元素，让你从内到外焕发光彩✨。无论是煲汤、炖煮还是凉拌，都能轻松驾驭，满足你对高端食材的所有幻想！

🦞 为什么选择海参？
- 高蛋白低脂肪，健康滋补两不误
- 提升免疫力，增强体质
- 美容养颜，延缓衰老

👏 无论是送长辈还是犒劳自己，海参都是不二之选！

😋 快来试试吧，让你的餐桌从此与众不同！

#海参滋补 #高端食材 #健康生活 #美容养颜 #深海珍品
####################################
## ✨【美妆种草】这款神仙产品让我秒变仙女！🧚‍♀️

姐妹们！今天我要分享一款让我惊艳到尖叫的美妆神器！🌸

最近入手了这款DGH3e，简直是化妆包里的宝藏！无论是日常通勤还是约会派对，它都能让我瞬间拥有水润透亮的肌肤，妆容服帖到不行！🌈

🌟 亮点：
- 超强持妆力，一整天不脱妆！
- 轻薄质地，完全不会闷痘！
- 提亮肤色，自带柔焦效果！

用了它之后，连闺蜜都追着问我用了什么！简直是美妆界的黑科技！🪞

如果你也想拥有完美妆容，赶紧入手吧！绝对不会后悔！💖

#美妆种草 #神仙产品 #化妆必备 #妆容持久 #护肤美妆
####################################
## 创业路上，勇敢追梦！🚀 从0到1的蜕变之旅

创业，是一场持续探索未知、突破自我、追求梦想的旅程！📊

在这个充满可能性的时代，我们每个人都可以成为自己命运的主宰者。无论是从零开始的小白，还是已经踏上征途的创业者，每一步都值得被记录和分享！🧑‍💻

💡 我的创业心得：
1. **勇敢尝试**：不要害怕失败，每一次跌倒都是成长的垫脚石。
2. **持续学习**：市场瞬息万变，唯有不断学习才能立于不败之地。
3. **团队合作**：找到志同道合的伙伴，一起披荆斩棘！🤝
4. **坚持初心**：无论多难，记住为什么出发。

创业的路上，没有捷径，但每一步都充满惊喜和收获！💸 你准备好迎接挑战了吗？

#创业 #追梦人 #从0到1 #创业心得 #勇敢前行

#创业 #追梦人 #从0到1 #创业心得 #勇敢前行
